In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!apt install git-lfs

In [ ]:
import numpy as np

In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForQuestionAnswering
from transformers import pipeline

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("squad")

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoTokenizer

# model_checkpoint = "bert-base-cased"
model_checkpoint = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
max_length = 384
stride = 128

def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    
    return inputs

In [ ]:
train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_datasets["train"].column_names)

len(raw_datasets["train"]), len(train_dataset)

  0%|          | 0/88 [00:00<?, ?ba/s]

(87599, 88567)

In [ ]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length")

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names)

len(raw_datasets["validation"]), len(validation_dataset)

(10570, 10790)

In [ ]:
model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/657M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForQuestionAnswering.

Some layers of TFRobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [ ]:
tf_train_dataset = train_dataset.to_tf_dataset(
    columns=[
        "input_ids",
        "start_positions",
        "end_positions",
        "attention_mask"
    ],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=16,
)
tf_eval_dataset = validation_dataset.to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

In [ ]:
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_epochs = 3
num_train_steps = len(tf_train_dataset) * num_train_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
# We're going to do validation afterwards, so no validation mid-training
model.fit(tf_train_dataset,  epochs=num_train_epochs)

Epoch 1/3
5536/5536 [==============================] - 1022s 181ms/step - loss: 1.0572
Epoch 2/3
5536/5536 [==============================] - 999s 180ms/step - loss: 0.7102
Epoch 3/3
5536/5536 [==============================] - 1001s 181ms/step - loss: 0.5554


In [ ]:
from tqdm.auto import tqdm
import collections
import evaluate

metric = evaluate.load("squad")

n_best = 20
max_answer_length = 30
predicted_answers = []

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)


In [ ]:
predictions = model.predict(tf_eval_dataset)

675/675 [==============================] - 55s 76ms/step


In [ ]:
predictions

TFQuestionAnsweringModelOutput(loss=None, start_logits=array([[-7.223308 , -9.265752 , -9.308378 , ..., -9.860869 , -9.860869 ,
        -9.860869 ],
       [-7.271986 , -9.25589  , -9.364266 , ..., -9.849108 , -9.849108 ,
        -9.849108 ],
       [-7.0323167, -9.33539  , -9.457763 , ..., -9.968013 , -9.968013 ,
        -9.968013 ],
       ...,
       [-7.596895 , -9.164059 , -9.442506 , ..., -9.794446 , -9.794446 ,
        -9.794446 ],
       [-7.7238293, -9.270277 , -9.219497 , ..., -9.702128 , -9.702128 ,
        -9.702128 ],
       [-7.742715 , -9.294847 , -9.425083 , ..., -9.653384 , -9.653384 ,
        -9.653384 ]], dtype=float32), end_logits=array([[-5.740149 , -9.382913 , -9.169973 , ..., -9.473579 , -9.473579 ,
        -9.473579 ],
       [-5.4049153, -9.384502 , -9.213943 , ..., -9.489263 , -9.489263 ,
        -9.489263 ],
       [-4.3993845, -9.285364 , -9.248752 , ..., -9.32376  , -9.32376  ,
        -9.32376  ],
       ...,
       [-6.6577234, -9.570117 , -9.407682 , ...

In [ ]:
compute_metrics(
    predictions["start_logits"],
    predictions["end_logits"],
    validation_dataset,
    raw_datasets["validation"])

  0%|          | 0/10570 [00:00<?, ?it/s]

{'exact_match': 85.59129612109744, 'f1': 92.14890304334105}

In [ ]:
%pwd

'/content'

In [ ]:
model.save_pretrained("/content/gdrive/MyDrive/Transformers_files/scratch_roberta_squad")
tokenizer.save_pretrained("/content/gdrive/MyDrive/Transformers_files/scratch_roberta_squad")

('/content/gdrive/MyDrive/Transformers_files/scratch_roberta_squad/tokenizer_config.json',
 '/content/gdrive/MyDrive/Transformers_files/scratch_roberta_squad/special_tokens_map.json',
 '/content/gdrive/MyDrive/Transformers_files/scratch_roberta_squad/vocab.json',
 '/content/gdrive/MyDrive/Transformers_files/scratch_roberta_squad/merges.txt',
 '/content/gdrive/MyDrive/Transformers_files/scratch_roberta_squad/added_tokens.json',
 '/content/gdrive/MyDrive/Transformers_files/scratch_roberta_squad/tokenizer.json')

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "/content/gdrive/MyDrive/Transformers_files/scratch_roberta_squad"
 
prueba_modelo= pipeline("question-answering", model=model_checkpoint)

All model checkpoint layers were used when initializing TFRobertaForQuestionAnswering.

All the layers of TFRobertaForQuestionAnswering were initialized from the model checkpoint at /content/gdrive/MyDrive/Transformers_files/scratch_roberta_squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForQuestionAnswering for predictions without further training.


In [ ]:

context = """
🤗 Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch and TensorFlow — with a seamless integration
between them. It's straightforward to train your models with one before loading them for inference with the other.
"""
question = "Which deep learning libraries back 🤗 Transformers?"
prueba_modelo(question=question, context=context)

{'score': 0.99609375,
 'start': 78,
 'end': 105,
 'answer': 'Jax, PyTorch and TensorFlow'}